In [3]:
"""
Preamble for most code and jupyter notebooks
@author: tobinsouth
@notebook date: 11 Nov 2021
"""

import numpy as np, pandas as pd, matplotlib.pyplot as plt, matplotlib as mpl, seaborn as sns
import math, string, re, pickle, json, os, sys, datetime, itertools
from collections import Counter
from tqdm import tqdm

# Set panda's options
pd.set_option("display.max_rows", 50)
pd.set_option("display.max_columns", 120)

# Better graphics
from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('retina')
plt.style.use('seaborn-poster')

In [4]:
data = pickle.load(open('../DeepMove/data/foursquare.pk', 'rb'),  encoding='latin')

FileNotFoundError: [Errno 2] No such file or directory: '../DeepMove/data/foursquare.pk'

In [ ]:
data.keys()

dict_keys(['data_filter', 'parameters', 'vid_lookup', 'vid_list', 'data_neural', 'uid_list'])

In [ ]:
all_stays_csvs = glob.glob(data_directory+'/*.csv.gz')

In [4]:
data.keys()

dict_keys(['data_filter', 'parameters', 'vid_lookup', 'vid_list', 'data_neural', 'uid_list'])

In [ ]:
need: history_loc
history_tim
history_count

In [19]:
data['data_neural'][0]['train']

[0, 1, 2, 3, 4]

In [20]:
import torch
from torch.autograd import Variable

data_neural = data['data_neural']
candidate = data_neural.keys()
mode = 'train'

data_train = {}
train_idx = {}
for u in candidate: # Users?
    sessions = data_neural[u]['sessions']
    train_id = data_neural[u][mode]
    data_train[u] = {}
    for c, i in enumerate(train_id):
        trace = {}
        if mode == 'train' and c == 0:
            continue
        session = sessions[i]
        target = np.array([s[0] for s in session[1:]])

        history = []
        # if mode == 'test':
        #     test_id = data_neural[u]['train']
        #     for tt in test_id:
        #         history.extend([(s[0], s[1]) for s in sessions[tt]])
        for j in range(c):
            history.extend([(s[0], s[1]) for s in sessions[train_id[j]]])

        history_tim = [t[1] for t in history]
        history_count = [1]
        last_t = history_tim[0]
        count = 1
        for t in history_tim[1:]:
            if t == last_t:
                count += 1
            else:
                history_count[-1] = count
                history_count.append(1)
                last_t = t
                count = 1

        history_loc = np.reshape(np.array([s[0] for s in history]), (len(history), 1))
        history_tim = np.reshape(np.array([s[1] for s in history]), (len(history), 1))
        trace['history_loc'] = Variable(torch.LongTensor(history_loc))
        trace['history_tim'] = Variable(torch.LongTensor(history_tim))
        trace['history_count'] = history_count

        loc_tim = history
        loc_tim.extend([(s[0], s[1]) for s in session[:-1]])
        loc_np = np.reshape(np.array([s[0] for s in loc_tim]), (len(loc_tim), 1))
        tim_np = np.reshape(np.array([s[1] for s in loc_tim]), (len(loc_tim), 1))
        trace['loc'] = Variable(torch.LongTensor(loc_np))
        trace['tim'] = Variable(torch.LongTensor(tim_np))
        trace['target'] = Variable(torch.LongTensor(target))
        data_train[u][i] = trace
    train_idx[u] = train_id

In [38]:
data_train[10][1]

{'history_loc': tensor([[437],
         [438],
         [439],
         [440],
         [441],
         [442],
         [440],
         [443],
         [441],
         [444],
         [445]]),
 'history_tim': tensor([[11],
         [12],
         [45],
         [46],
         [ 0],
         [20],
         [ 9],
         [10],
         [20],
         [11],
         [20]]),
 'history_count': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'loc': tensor([[437],
         [438],
         [439],
         [440],
         [441],
         [442],
         [440],
         [443],
         [441],
         [444],
         [445],
         [437],
         [ 15],
         [446],
         [446]]),
 'tim': tensor([[11],
         [12],
         [45],
         [46],
         [ 0],
         [20],
         [ 9],
         [10],
         [20],
         [11],
         [20],
         [34],
         [35],
         [36],
         [41]]),
 'target': tensor([ 15, 446, 446, 447])}

In [65]:
from glob import glob
all_csvs = glob('../../data/.each_traj/traj_*.csv')
with open('data/geoid_mapping.pickle', "rb") as f:
    geoid_mapping = pickle.load(f)

# all_moribus_data = {}
# for u, file in tqdm(enumerate(all_csvs[:10000])):
#     data = pd.read_csv(file)
#     data['tim'] = np.floor(data['time']).astype(int)
#     data['geoid'] = data['GEOID'].map(geoid_mapping).fillna(0).astype(int)

#     home = data['geoid'].value_counts().index[0]
#     this_users_traces = {}
#     trace_counter = 0
#     geoids, tims, geoids_history, tims_history = [],[]
#     for i, (geoid, tim) in enumerate(zip(data['geoid'], data['tim'])):
#         geoids.append(geoid)
#         tims.append(tim)

#         if i > 5 and geoid == home:
#             if len(tims_history) != 0:
#                this_users_traces[trace_counter] = {'history_loc': geoids_history, 'history_tim': tims_history, 'history_count': [1]*len(geoids_history), 'loc': geoids[:-1], 'tim': tims[:-1], 'target':geoids[-1]}
#                trace_counter+=1
#             geoids_history = geoids
#             geoids = []
#             tims_history = tims
#             tims = []

#     all_moribus_data[u] = this_users_traces


In [58]:
all_moribus_data

{}

In [54]:
trace

{'history_loc': tensor([[  165],
         [  165],
         [10491],
         [10491],
         [ 5278],
         [  460],
         [ 1358],
         [  225],
         [  165],
         [10492],
         [  165],
         [10493],
         [  165],
         [  165],
         [  165],
         [ 4674],
         [  165],
         [10491],
         [  105],
         [ 2890],
         [10493],
         [  165],
         [10491],
         [ 5278],
         [  465],
         [  105],
         [  165],
         [ 6245],
         [ 4674],
         [ 2890],
         [  905],
         [  105],
         [  165],
         [  165],
         [10494],
         [10495],
         [ 3189],
         [ 1220],
         [ 5541],
         [10491],
         [  165],
         [  165],
         [10494],
         [ 6245],
         [10493],
         [ 4674],
         [  165],
         [  165],
         [ 4674],
         [  165],
         [  165],
         [10492],
         [  105],
         [  165],
         [  1